In [ ]:
import os
import time
from datetime import datetime, timedelta
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# ================= CONFIG =================

URL = "https://www.fpi.nsdl.co.in/web/Reports/Archive.aspx"

START_YEAR = 2025
CUTOFF_YEAR = 2026
END_DATE = datetime.today()

DOWNLOAD_DIR = os.path.abspath("downloads")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# ================= SELENIUM SETUP =================

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option(
    "prefs",
    {
        "download.default_directory": DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "directory_upgrade": True,
        "safebrowsing.enabled": True,
    },
)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# ================= HELPERS =================

def wait_and_rename_file(report_date, before_files, timeout=60):
    end_time = time.time() + timeout
    while time.time() < end_time:
        after_files = set(os.listdir(DOWNLOAD_DIR))
        new_files = after_files - before_files

        for f in new_files:
            if f.endswith((".csv", ".xls")):
                old_path = os.path.join(DOWNLOAD_DIR, f)
                ext = os.path.splitext(f)[1]
                new_path = os.path.join(
                    DOWNLOAD_DIR, f"nsdl_fpi_{report_date}{ext}"
                )
                os.rename(old_path, new_path)
                return new_path

        time.sleep(1)

    raise TimeoutError("Download not detected")

def set_date_and_download(date_obj):
    date_ui = date_obj.strftime("%d-%b-%Y")
    date_file = date_obj.strftime("%Y-%m-%d")
    print("Fetching:", date_ui)

    before_files = set(os.listdir(DOWNLOAD_DIR))

    date_input = driver.find_element(By.ID, "txtDate")
    driver.execute_script("arguments[0].removeAttribute('disabled')", date_input)
    date_input.clear()
    date_input.send_keys(date_ui)

    driver.execute_script(
        "document.getElementById('hdnDate').value = arguments[0]",
        date_ui
    )

    driver.find_element(By.ID, "btnSubmit1").click()
    time.sleep(3)

    driver.find_element(By.ID, "btnExcel").click()

    file_path = wait_and_rename_file(date_file, before_files)

    if file_path.endswith(".csv"):
        return pd.read_csv(file_path)
    else:
        return pd.read_excel(file_path)

# ================= SCRAPING =================

driver.get(URL)
time.sleep(3)

all_equity_data = []

# -------- 2000–2009 : monthly last trading day --------

for year in range(START_YEAR, CUTOFF_YEAR):
    for month in range(1, 13):
        last_day = datetime(year, month, 1) + timedelta(days=32)
        last_day = last_day.replace(day=1) - timedelta(days=1)

        try:
            df = set_date_and_download(last_day)
            equity_df = df[df["Debt/Equity"].str.contains("Equity", na=False)].copy()
            all_equity_data.append(equity_df)
        except Exception as e:
            print(f"Failed {year}-{month:02d}: {e}")

# -------- 2010–today : WEEKLY (Fridays only) --------
"""
current = datetime(CUTOFF_YEAR, 1, 1)

while current <= END_DATE:
    if current.weekday() == 4:  # Friday
        try:
            df = set_date_and_download(current)
            equity_df = df[df["Debt/Equity"].str.contains("Equity", na=False)].copy()
            all_equity_data.append(equity_df)
        except Exception as e:
            print(f"Failed {current.strftime('%Y-%m-%d')}: {e}")

    current += timedelta(days=1)
"""
# ================= FINAL CLEAN =================

driver.quit()

if not all_equity_data:
    raise RuntimeError("No data collected")

final_df = pd.concat(all_equity_data, ignore_index=True)

final_df.drop_duplicates(
    subset=["Reporting Date", "Debt/Equity", "Investment Route"],
    inplace=True
)

final_df.sort_values("Reporting Date", inplace=True)
final_df.to_csv("fii_equity_2000_to_today.csv", index=False)

print("Done. Saved to fii_equity_2000_to_today.csv")


2026-02-10 16:06:04,320 - INFO - ====== WebDriver manager ======
2026-02-10 16:06:05,315 - INFO - Get LATEST chromedriver version for google-chrome
2026-02-10 16:06:05,823 - INFO - Get LATEST chromedriver version for google-chrome
2026-02-10 16:06:06,318 - INFO - Driver [C:\Users\patel\.wdm\drivers\chromedriver\win64\144.0.7559.133\chromedriver-win32/chromedriver.exe] found in cache


Fetching: 31-Jan-2025
Failed 2025-01: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 28-Feb-2025
Failed 2025-02: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 31-Mar-2025
Failed 2025-03: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 30-Apr-2025
Failed 2025-04: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 31-May-2025
Failed 2025-05: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 30-Jun-2025
Failed 2025-06: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 31-Jul-2025
Failed 2025-07: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 31-Aug-2025
Failed 2025-08: Excel file format cannot be determined, you must specify an engine manually.
Fetching: 30-Sep-2025
Failed 2025-09: Excel file format cannot be determined, yo

RuntimeError: No data collected

In [21]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for glob


In [34]:
import pandas as pd
import glob
import os

# 1. Set your specific Windows folder path
# The 'r' before the string handles the Windows backslashes correctly
path = r"C:\Users\patel\OneDrive\Desktop\Code\downloads\before 2010"
file_list = glob.glob(os.path.join(path, "*.xls"))

all_dfs = []

for file_path in file_list:
    try:
        # Use read_html as these are actually HTML files with .xls extensions
        tables = pd.read_html(file_path)
        df_investments = tables[0]
        
        # Hardcode the column names (adjusted for the usual artifact column in these NSDL files)
        num_cols = len(df_investments.columns)
        headers = [
            'Reporting Date', 
            'Debt/Equity', 
            'Gross Purchases(Rs Crore)', 
            'Gross Sales(Rs Crore)', 
            'Net Investment (Rs Crore)', 
            'Net Investment US($) million', 
            'Conversion (1 USD TO INR)',
            'nan'
        ]
        
        # If the file has a 9th empty column (common in these scrapes), name it 'nan'
        if num_cols > len(headers):
            headers += ['nan'] * (num_cols - len(headers))
        
        df_investments.columns = headers
        
        # Cleaning logic based on your code
        # We slice from index 3 to skip the HTML disclaimer and header rows
        df_clean = df_investments.iloc[0:].reset_index(drop=True)
        
        # Fill the merged date cells
        df_clean['Reporting Date'] = df_clean['Reporting Date'].ffill()
        
        # Filter for Equity only
        df_equity_only = df_clean[df_clean['Debt/Equity'] == 'Equity'].copy()
        
        # Remove the bottom 3 rows (Sub-totals/Notes)
    #    df_equity_only = df_equity_only.iloc[:-3]
        # 1. Clean the 'Reporting Date' column of hidden junk (newlines, tabs, extra spaces)
        df_equity_only['Reporting Date'] = df_equity_only['Reporting Date'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

        # 2. Convert to datetime, but COERCE errors to NaN
        # This turns "Total for Month", "Notes", etc. into NaN
        valid_dates = pd.to_datetime(df_equity_only['Reporting Date'], errors='coerce')

        # 3. Filter: Keep only rows where 'Reporting Date' is NOT NaN
        df_equity_only = df_equity_only[valid_dates.notna()].copy()
            
        all_dfs.append(df_equity_only)
        print(f"Processed: {os.path.basename(file_path)}")
        
    except Exception as e:
        print(f"Skipping {os.path.basename(file_path)} due to error: {e}")

# 2. Combine all files into one master DataFrame
if all_dfs:
    final_combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\nSuccessfully combined {len(all_dfs)} files!")
    
    # Optional: Save the final result to your desktop
    # final_combined_df.to_csv(os.path.join(path, "combined_equity_data.csv"), index=False)
else:
    print("No files were processed. Check if the folder path is correct and contains .xls files.")

final_combined_df.head(150)

Processed: nsdl_fpi_2000-01-31.xls
Processed: nsdl_fpi_2000-02-29.xls
Processed: nsdl_fpi_2000-03-31.xls
Processed: nsdl_fpi_2000-04-30.xls
Processed: nsdl_fpi_2000-05-31.xls
Processed: nsdl_fpi_2000-06-30.xls
Processed: nsdl_fpi_2000-07-31.xls
Processed: nsdl_fpi_2000-08-31.xls
Processed: nsdl_fpi_2000-09-30.xls
Processed: nsdl_fpi_2000-10-31.xls
Processed: nsdl_fpi_2000-11-30.xls
Processed: nsdl_fpi_2000-12-31.xls
Processed: nsdl_fpi_2001-01-31.xls
Processed: nsdl_fpi_2001-02-28.xls
Processed: nsdl_fpi_2001-03-31.xls
Processed: nsdl_fpi_2001-04-30.xls
Processed: nsdl_fpi_2001-05-31.xls
Processed: nsdl_fpi_2001-06-30.xls
Processed: nsdl_fpi_2001-07-31.xls
Processed: nsdl_fpi_2001-08-31.xls
Processed: nsdl_fpi_2001-09-30.xls
Processed: nsdl_fpi_2001-10-31.xls
Processed: nsdl_fpi_2001-11-30.xls
Processed: nsdl_fpi_2001-12-31.xls
Processed: nsdl_fpi_2002-01-31.xls
Processed: nsdl_fpi_2002-02-28.xls
Processed: nsdl_fpi_2002-03-31.xls
Processed: nsdl_fpi_2002-04-30.xls
Processed: nsdl_fpi_

,Reporting Date,Debt/Equity,Gross Purchases(Rs Crore),Gross Sales(Rs Crore),Net Investment (Rs Crore),Net Investment US($) million,Conversion (1 USD TO INR),nan
0,04-Jan-2000,Equity,148.40,59.30,89.10,20.50,Rs. 43.52,NaN
1,05-Jan-2000,Equity,333.40,238.20,95.20,21.90,Rs. 43.50,NaN
2,06-Jan-2000,Equity,250.30,528.30,(278.00),(63.90),Rs. 43.52,NaN
3,07-Jan-2000,Equity,269.00,559.70,(290.70),(66.80),Rs. 43.52,NaN
4,10-Jan-2000,Equity,259.50,223.10,36.40,8.40,Rs. 43.51,NaN
...,...,...,...,...,...,...,...,...
145,02-Aug-2000,Equity,158.20,236.00,(77.80),(17.20),Rs. 45.13,NaN
146,03-Aug-2000,Equity,84.00,90.00,(6.00),(1.30),Rs. 45.34,NaN
147,04-Aug-2000,Equity,287.90,241.20,46.70,10.30,Rs. 45.44,NaN
148,07-Aug-2000,Equity,269.70,163.80,105.90,23.10,Rs. 45.79,NaN


In [35]:
# Define the output file name
output_filename = "combined_equity_data_2009.csv"
output_path = os.path.join("C:/Users/patel/OneDrive/Desktop/Code/fpi_data", output_filename)

# Save the combined DataFrame
if not final_combined_df.empty:
    # index=False prevents pandas from adding an extra column with row numbers
    final_combined_df.to_csv(output_path, index=False)
    
    # Alternatively, for Excel:
    # final_combined_df.to_excel(output_path.replace(".csv", ".xlsx"), index=False)
    
    print(f"--- SUCCESS ---")
    print(f"File saved to: {output_path}")
    print(f"Total rows saved: {len(final_combined_df)}")
else:
    print("DataFrame is empty. Nothing to save.")

--- SUCCESS ---
File saved to: C:/Users/patel/OneDrive/Desktop/Code/fpi_data\combined_equity_data_2009.csv
Total rows saved: 2462


In [40]:
import pandas as pd
import glob
import os

# 1. Set your specific Windows folder path
# The 'r' before the string handles the Windows backslashes correctly
path = r"C:\Users\patel\OneDrive\Desktop\Code\downloads"
file_list = glob.glob(os.path.join(path, "*.xls"))

all_dfs = []

for file_path in file_list:
    try:
        # Use read_html as these are actually HTML files with .xls extensions
        tables = pd.read_html(file_path)
        df_investments = tables[0]
        
        # Hardcode the column names (adjusted for the usual artifact column in these NSDL files)
        num_cols = len(df_investments.columns)
        headers = [
            'Reporting Date', 
            'Debt/Equity', 
            'Investment Route', 
            'Gross Purchases(Rs Crore)', 
            'Gross Sales(Rs Crore)', 
            'Net Investment (Rs Crore)', 
            'Net Investment US($) million', 
            'Conversion (1 USD TO INR)',
            'nan'
        ]
        
        # If the file has a 9th empty column (common in these scrapes), name it 'nan'
        if num_cols > len(headers):
            headers += ['nan'] * (num_cols - len(headers))
        
        df_investments.columns = headers
        
        # Cleaning logic based on your code
        # We slice from index 3 to skip the HTML disclaimer and header rows
        df_clean = df_investments.iloc[0:].reset_index(drop=True)
        
        # Fill the merged date cells
        df_clean['Reporting Date'] = df_clean['Reporting Date'].ffill()
        
        # Filter for Equity only
        df_equity_only = df_clean[df_clean['Debt/Equity'] == 'Equity'].copy()
        
        # Remove the bottom 3 rows (Sub-totals/Notes)
    #    df_equity_only = df_equity_only.iloc[:-3]
        # 1. Clean the 'Reporting Date' column of hidden junk (newlines, tabs, extra spaces)
        df_equity_only['Reporting Date'] = df_equity_only['Reporting Date'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

        # 2. Convert to datetime, but COERCE errors to NaN
        # This turns "Total for Month", "Notes", etc. into NaN
        valid_dates = pd.to_datetime(df_equity_only['Reporting Date'], errors='coerce')

        # 3. Filter: Keep only rows where 'Reporting Date' is NOT NaN
        df_equity_only = df_equity_only[valid_dates.notna()].copy()
            
        all_dfs.append(df_equity_only)
        print(f"Processed: {os.path.basename(file_path)}")
        
    except Exception as e:
        print(f"Skipping {os.path.basename(file_path)} due to error: {e}")

# 2. Combine all files into one master DataFrame
if all_dfs:
    final_combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"\nSuccessfully combined {len(all_dfs)} files!")
    
    # Optional: Save the final result to your desktop
    # final_combined_df.to_csv(os.path.join(path, "combined_equity_data.csv"), index=False)
else:
    print("No files were processed. Check if the folder path is correct and contains .xls files.")

final_combined_df.head(150)

Processed: nsdl_fpi_2009-12-31.xls
Processed: nsdl_fpi_2010-01-31.xls
Processed: nsdl_fpi_2010-02-28.xls
Processed: nsdl_fpi_2010-03-31.xls
Processed: nsdl_fpi_2010-04-30.xls
Processed: nsdl_fpi_2010-05-31.xls
Processed: nsdl_fpi_2010-06-30.xls
Processed: nsdl_fpi_2010-07-31.xls
Processed: nsdl_fpi_2010-08-31.xls
Processed: nsdl_fpi_2010-09-30.xls
Processed: nsdl_fpi_2010-10-31.xls
Processed: nsdl_fpi_2010-11-30.xls
Processed: nsdl_fpi_2010-12-31.xls
Processed: nsdl_fpi_2011-01-31.xls
Processed: nsdl_fpi_2011-02-28.xls
Processed: nsdl_fpi_2011-03-31.xls
Processed: nsdl_fpi_2011-04-30.xls
Processed: nsdl_fpi_2011-05-31.xls
Processed: nsdl_fpi_2011-06-30.xls
Processed: nsdl_fpi_2011-07-31.xls
Processed: nsdl_fpi_2011-08-31.xls
Processed: nsdl_fpi_2011-09-30.xls
Processed: nsdl_fpi_2011-10-31.xls
Processed: nsdl_fpi_2011-11-30.xls
Processed: nsdl_fpi_2011-12-31.xls
Processed: nsdl_fpi_2012-01-31.xls
Processed: nsdl_fpi_2012-02-29.xls
Processed: nsdl_fpi_2012-03-31.xls
Processed: nsdl_fpi_

,Reporting Date,Debt/Equity,Investment Route,Gross Purchases(Rs Crore),Gross Sales(Rs Crore),Net Investment (Rs Crore),Net Investment US($) million,Conversion (1 USD TO INR),nan
0,01-Dec-2009,Equity,Stock Exchange,2812.30,2274.70,537.50,115.64,Rs.46.4800,NaN
1,01-Dec-2009,Equity,Primary market & others,163.80,1.50,162.30,34.92,Rs.46.4800,NaN
2,01-Dec-2009,Equity,Sub-total,2976.10,2276.20,699.80,150.57,Rs.46.4800,NaN
3,02-Dec-2009,Equity,Stock Exchange,2987.30,2237.80,749.50,161.35,Rs.46.4500,NaN
4,02-Dec-2009,Equity,Primary market & others,1561.50,1.00,1560.40,335.94,Rs.46.4500,NaN
...,...,...,...,...,...,...,...,...,...
145,11-Feb-2010,Equity,Primary market & others,13.60,28.70,(15.10),(3.24),Rs.46.5600,NaN
146,11-Feb-2010,Equity,Sub-total,2376.80,2400.40,(23.60),(5.07),Rs.46.5600,NaN
147,15-Feb-2010,Equity,Stock Exchange,1775.60,1457.90,317.60,68.37,Rs.46.4600,NaN
148,15-Feb-2010,Equity,Primary market & others,33.30,0.00,33.30,7.17,Rs.46.4600,NaN


In [41]:
# Define the output file name
output_filename = "combined_equity_data_2010.csv"
output_path = os.path.join("C:/Users/patel/OneDrive/Desktop/Code/fpi_data", output_filename)

# Save the combined DataFrame
if not final_combined_df.empty:
    # index=False prevents pandas from adding an extra column with row numbers
    final_combined_df.to_csv(output_path, index=False)
    
    # Alternatively, for Excel:
    # final_combined_df.to_excel(output_path.replace(".csv", ".xlsx"), index=False)
    
    print(f"--- SUCCESS ---")
    print(f"File saved to: {output_path}")
    print(f"Total rows saved: {len(final_combined_df)}")
else:
    print("DataFrame is empty. Nothing to save.")

--- SUCCESS ---
File saved to: C:/Users/patel/OneDrive/Desktop/Code/fpi_data\combined_equity_data_2010.csv
Total rows saved: 11685


In [ ]:
import pandas as pd

# Use read_html instead of read_excel
file_path = "/content/nsdl_fpi_2009-11-30.xls"
tables = pd.read_html(file_path)

# tables is a list. Based on your data:
# tables[0] should be the FPI Investment table
# tables[1] should be the FPI Derivative Trades table

df_investments = tables[0]
df_clean = df_investments
df_clean.columns = df_clean.iloc[0] # Set the first row as columns
# Hardcode the column names exactly as they appear in the table
df_investments.columns = [
    'Reporting Date',
    'Debt/Equity',
    'Investment Route',
    'Gross Purchases(Rs Crore)',
    'Gross Sales(Rs Crore)',
    'Net Investment (Rs Crore)',
    'Net Investment US($) million',
    'Conversion (1 USD TO INR)',
 #   'nan'
]

# Drop the first 3 rows (disclaimer text and the original header rows)
df_clean = df_investments.iloc[0:].reset_index(drop=True)
#df_clean.drop(columns=['nan'], inplace=True)
df_clean['Reporting Date'] = df_clean['Reporting Date'].ffill()
df_equity_only = df_clean[df_clean['Debt/Equity'] == 'Equity'].copy()
df_equity_only = df_equity_only.iloc[:-3]
df_equity_only.head(150)


In [ ]:
import pandas as pd

# Load your file
df = pd.read_csv('/content/combined_equity_data_2010.csv')

# Filter the dataframe
# We use == to keep only the 'Stock Exchange' entries
filtered_df = df[df['Investment Route'] == 'Stock Exchange']

# Save the cleaned version
#filtered_df.to_csv('filtered_data.csv', index=False)

filtered_df.head(100)

filtered_df = filtered_df.drop(columns=['Investment Route'])

filtered_df.head(100)

In [ ]:
# 2. Load your existing CSV file
# Replace this path if your file is located elsewhere
other_df_path = '/content/combined_equity_data_2009.csv'
other_df = pd.read_csv(other_df_path)

# 3. Combine (concatenate) the two DataFrames
# ignore_index=True ensures the row numbers are reset and continuous
final_combined_df = pd.concat([other_df, filtered_df], ignore_index=True)

# 4. Save the combined result
final_combined_df.to_csv('combined_fpi_equity_data.csv', index=False)

print("Data combined successfully! First 5 rows of the new dataset:")
print(final_combined_df.head())